In [2]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
import re
import nltk as nlp

import os
import matplotlib.pyplot as plt
import  nltk
%matplotlib inline
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package stopwords to /home/kapil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
train_data=pd.read_csv("/home/kapil/Downloads/0000000000002747_training_twitter_x_y_train.csv")
len(train_data)


10980

In [5]:
train_data.shape

(10980, 12)

In [54]:
train_data['airline'].value_counts()

United            2928
US Airways        2152
American          2078
Southwest         1817
Delta             1639
Virgin America     366
Name: airline, dtype: int64

In [83]:
train_data['retweet_count'].value_counts()

0     10410
1       479
2        45
4        17
3        15
5         3
7         2
6         2
31        1
15        1
22        1
44        1
28        1
18        1
8         1
Name: retweet_count, dtype: int64

In [18]:
train_data.head(5)

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [27]:
train_data.isna().sum()

tweet_id                      0
airline_sentiment             0
airline                       0
airline_sentiment_gold    10949
name                          0
negativereason_gold       10956
retweet_count                 0
text                          0
tweet_coord               10204
tweet_created                 0
tweet_location             3550
user_timezone              3577
dtype: int64

In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10980 entries, 0 to 10979
Data columns (total 12 columns):
tweet_id                  10980 non-null int64
airline_sentiment         10980 non-null object
airline                   10980 non-null object
airline_sentiment_gold    31 non-null object
name                      10980 non-null object
negativereason_gold       24 non-null object
retweet_count             10980 non-null int64
text                      10980 non-null object
tweet_coord               776 non-null object
tweet_created             10980 non-null object
tweet_location            7430 non-null object
user_timezone             7403 non-null object
dtypes: int64(2), object(10)
memory usage: 1.0+ MB


In [6]:
train_data.keys()

Index(['tweet_id', 'airline_sentiment', 'airline', 'airline_sentiment_gold',
       'name', 'negativereason_gold', 'retweet_count', 'text', 'tweet_coord',
       'tweet_created', 'tweet_location', 'user_timezone'],
      dtype='object')

In [7]:
train_data.airline_sentiment

0        negative
1        positive
2        positive
3        negative
4        negative
5        negative
6        negative
7        positive
8        negative
9        positive
10       negative
11        neutral
12       positive
13       negative
14       negative
15       negative
16       negative
17       negative
18       negative
19       negative
20       negative
21        neutral
22       negative
23       negative
24       positive
25        neutral
26       negative
27       negative
28       positive
29       positive
           ...   
10950    positive
10951    negative
10952    negative
10953    negative
10954    negative
10955    negative
10956     neutral
10957    negative
10958    negative
10959    negative
10960    negative
10961    negative
10962     neutral
10963    negative
10964    negative
10965     neutral
10966    negative
10967    negative
10968    negative
10969    positive
10970    negative
10971     neutral
10972    negative
10973    negative
10974    p

In [29]:
neg_train_data = train_data.loc[train_data['airline_sentiment']=='negative']
neg_train_data.isna().sum()

tweet_id                     0
airline_sentiment            0
airline                      0
airline_sentiment_gold    6827
name                         0
negativereason_gold       6827
retweet_count                0
text                         0
tweet_coord               6356
tweet_created                0
tweet_location            2359
user_timezone             2372
dtype: int64

In [30]:
pos_train_data = train_data.loc[train_data['airline_sentiment']=='positive']
pos_train_data.isna().sum()

tweet_id                     0
airline_sentiment            0
airline                      0
airline_sentiment_gold    1798
name                         0
negativereason_gold       1802
retweet_count                0
text                         0
tweet_coord               1665
tweet_created                0
tweet_location             474
user_timezone              496
dtype: int64

In [85]:
train_data['airline_sentiment'].value_counts()

negative    6851
neutral     2327
positive    1802
Name: airline_sentiment, dtype: int64

In [32]:
dataset=train_data.text.dropna()
print('Number of observations are: '+str(len(dataset)))

Number of observations are: 10980


In [33]:
train_data['tweet_length'] = train_data['text'].apply(lambda x : len(x))
train_data['num_hashtags'] = train_data['text'].apply(lambda x : len(re.compile(r"#(\w+)").findall(x)))
train_data['num_mentions'] = train_data['text'].apply(lambda x : len(re.compile(r"@(\w+)").findall(x)))

In [35]:
mean_train_data = train_data.groupby('airline_sentiment').mean()
mean_train_data[['tweet_length','num_hashtags', 'num_mentions']]

,tweet_length,num_hashtags,num_mentions
airline_sentiment,,,
negative,114.167129,0.231353,1.110787
neutral,87.078212,0.209282,1.172755
positive,85.145949,0.283019,1.133185


In [92]:
sentiments=train_data.airline_sentiment
text=train_data.text

In [93]:
documents=[]
for i in range(len(sentiments)):
    documents.append((text[i],sentiments[i]))
documents[0]

('@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled',
 'negative')

In [8]:
stop = stopwords.words('english')
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [58]:
from nltk.corpus import stopwords
from string import punctuation

In [59]:
import string
stops=set(stopwords.words('english'))
punctuations=list(string.punctuation)
stops.update(punctuations)
stops

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'need

In [60]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [61]:
def simplify_pos_tag(tag):
    if tag.startswith('N'):
        return 'n'
    if tag.startswith('J'):
        return 'a'
    if tag.startswith('R'):
        return 'r'
    if tag.startswith('V'):
        return 'v'
    return 'n'

In [14]:
dataset = train_data[["airline_sentiment","text"]]
dataset.head(7)

,airline_sentiment,text
0,negative,"@SouthwestAir I am scheduled for the morning, ..."
1,positive,@SouthwestAir seeing your workers time in and ...
2,positive,@united Flew ORD to Miami and back and had gr...
3,negative,@SouthwestAir @dultch97 that's horse radish 😤🐴
4,negative,@united so our flight into ORD was delayed bec...
5,negative,@united Why did you load us in this flying sar...
6,negative,@JetBlue that is a stock response. Delays not ...


In [62]:
sentiments ={"neutral":0,
             "positive":1,
             "negative":-1}
dataset.airline_sentiment = [sentiments[i] for i in train_data.airline_sentiment]
dataset.head()


0    @SouthwestAir I am scheduled for the morning, ...
1    @SouthwestAir seeing your workers time in and ...
2    @united Flew ORD to Miami and back and  had gr...
3       @SouthwestAir @dultch97 that's horse radish 😤🐴
4    @united so our flight into ORD was delayed bec...
Name: text, dtype: object

In [63]:
def hasNumbers(text):
    return bool(re.search('[0-9]+',text))

In [64]:
def clean_text(text):
    result=[]
    for word in text.split():
        word=word.strip(punctuation)
        if ((word.lower() not in stops) and (not hasNumbers(word.lower())) and (len(word.lower())>3)):
            posTag=pos_tag([word.lower()])[0][1]
            lemmatized_word=lemmatizer.lemmatize(word.lower(),simplify_pos_tag(posTag))
            result.append(lemmatized_word)
    return result

In [71]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
 
documents_modified=[(clean_text(text),sentiment) for text,sentiment in documents]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kapil/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/kapil/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [73]:
test_data=pd.read_csv("/home/kapil/Downloads/0000000000002747_test_twitter_x_test.csv")

In [74]:
x_test=test_data.text
x_test=[clean_text(text) for text in x_test]

In [75]:
y_train = [sentiment for text,sentiment in documents_modified]
x_train = [" ".join(text) for text,sentiment in documents_modified]
x_test = [" ".join(text) for text in x_test]

In [76]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [77]:
count_vec=CountVectorizer(max_features=15000,ngram_range=(1,2))
x_train_features = count_vec.fit_transform(x_train)
x_test_features = count_vec.transform(x_test)
clf=MultinomialNB()
clf.fit(x_train_features,y_train)
pred=clf.predict(x_test_features)

In [78]:
clf.score(x_train_features,y_train)

0.8682149362477232

In [84]:
np.savetxt("twitter_sentiments_predictions.csv", pred , fmt = '%s')